## Self study 6

In this self study we directly continue with what we started in self study 5

**Task 2:** For the same user, and the same train/test split, implement the user-based neighborhood method. How does the RMSE compare to what you got in the content-based approach?

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
#Load data
R=np.genfromtxt('u.data',dtype=int)

#Sort users per number of ratings
Nratings=np.zeros(943)
for i in range(943):
    Nratings[i]=len(np.where(R[:,0]==i+1)[0])

#Get users with most ratings and make train/test split
users = np.where(Nratings>500)
ratID = np.where(R[::,0] == users[0][3]+1)
ratings = [R[i, 1:3] for i in ratID[0]]

split = len(ratings) // 3
train = pd.DataFrame(ratings[:2*split], columns=["Movie ID", "Rating"])
test = pd.DataFrame(ratings[2*split:], columns=["Movie ID", "Rating"])


In [22]:
# Make a prediction for a movie
def estimate_rating(train, movie, user):
    # Get all user which have rated the movie
    users = np.where(R[::,1] == movie)
    # Perform mean centering on original user vector
    avg_rating = np.average(train["Rating"])
    train["Relative Rating"] = train["Rating"] - avg_rating
    users_with_ratings = []
    # Extract user IDs
    users = [R[i,0] for i in users[0] if i != user]
    for us in users:
        # Extract the ratings of each user
        ratIDs = np.where(R[::,0] == us)[0]
        ratings = np.asarray([R[i,1:3] for i in ratIDs])
        # Perfrom mean centering on user vector
        user_avg = np.average(ratings[::,1])
        ratings = np.c_[ratings, [row[1] - user_avg for row in ratings]]
        # Find common ratings
        common_movies = [int(i) for i in ratings[::,0] if i in train["Movie ID"].values]
        common_movies.sort()
        # Construct user vectors of common movies
        old_user_ratings = np.asarray([train.values[[np.where(train["Movie ID"].values == mov)[0]], 2][0] for mov in common_movies])[::,0]
        new_user_ratings = np.asarray([ratings[np.where(ratings[::,0] == mov)[0], 2] for mov in common_movies])[::,0]
        # Calculate cosine similarity with significance
        sim = np.dot(old_user_ratings, new_user_ratings)/ (np.linalg.norm(old_user_ratings) * np.linalg.norm(new_user_ratings))
        sim *= min([len(common_movies), 50]) / 50

        users_with_ratings.append((sim, ratings[np.where(ratings[::,0] == movie)[0][0],2]))
    # Sort users by similarity and choose top 50
    users_with_ratings.sort(reverse=True, key= lambda x: x[0])
    users_with_ratings = users_with_ratings[:50]
    result = [x[1] for x in users_with_ratings]
    return np.average(result) + avg_rating

In [23]:
rats = []
for m in test.values:
    r = estimate_rating(train, m[0], users[0][3]+1)
    rats.append(r)
rats = np.asarray(rats)

truth = np.asarray(test['Rating'])
RMSE = math.sqrt(np.average((truth-rats) ** 2))
RMSE

0.7280623261159056

**Task 3:** Now take the whole rating matrix. Divide all the ratings in the matrix into a train and a test set. For the user you have used in Tasks 1 and 2, you should include the same ratings in the test set as before. Be sure that you do the train/test division uniformly at random over all ratings, so that you don't get all the ratings of one movie (or one user) included in the test set (that would lead to some cold start problems).  You can also start with a sub-matrix of the whole rating matrix by selecting only a subset of users, and a subset of movies.

Implement the gradient-descent approach for minimizing the error function for the ratings in the training set (you have to pick a value for the number of latent dimensions). Compare the RMSE values you obtain for the test user of tasks 1 and 2 with what you have previously obtained.  